In [34]:
import os
import random

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace,rand,col
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, TimestampType

import warnings

# Exercise 1

In [30]:
def calculate_trailing_zeros(hash_function, value):
    result = hash_function(value)
    bit_array = f'{result:05b}'
    for i in range(0,5):
        if bit_array[4-i] == '1':
            return i

In [33]:
input_stream = [3,1,4,1,5,9,2,6,5]
hash_func_1 = lambda x : (2*x + 1) % 32
hash_func_2 = lambda x : (3*x + 7) % 32
hash_func_3 = lambda x : (4*x) % 32

values_func_1=[]
values_func_2=[]
values_func_3=[]

for x in input_stream:
    values_func_1.append(calculate_trailing_zeros(hash_func_1,x))
    values_func_2.append(calculate_trailing_zeros(hash_func_2,x))
    values_func_3.append(calculate_trailing_zeros(hash_func_3,x))

print("Results for function 1")
print(values_func_1)
print("Results for function 2")
print(values_func_2)
print("Results for function 3")
print(values_func_3)

Results for function 1
[0, 0, 0, 0, 0, 0, 0, 0, 0]
Results for function 2
[4, 1, 0, 1, 1, 1, 0, 0, 1]
Results for function 3
[2, 2, 4, 2, 2, 2, 3, 3, 2]


# Bonus point question:
By using mod 2^x one cuts of the upper part of the binary representation of the bit array produced initially.
Therefore, the mod does not impact the spread of the values, but only the initial function has any impact.

# Exercise 2

We have the following input stream: 3,4,1,3,4,2,1,2

Let's now construct the Xi's:
$X_{1}: {X_{1}.el=3, X_{1}.val=2}$
$X_{2}: {X_{2}.el=4, X_{2}.val=2}$
$X_{3}: {X_{3}.el=1, X_{3}.val=2}$
$X_{4}: {X_{4}.el=3, X_{4}.val=1}$
$X_{5}: {X_{5}.el=4, X_{5}.val=1}$
$X_{6}: {X_{6}.el=2, X_{6}.val=2}$
$X_{7}: {X_{7}.el=1, X_{7}.val=1}$
$X_{8}: {X_{8}.el=2, X_{8}.val=1}$

No because then we could simply count the momentum for each different element.
The AMS algorithm is used when there is not enough storage to do so.

# Exercise 3

In [90]:
# n = length of the stream, k = degree of the moment(valid values:1,2,3), v = amount of variables to use
def compute_moment_ams(input_stream, n, k, v):

    if k<1 or k > 3 or v > n:
        print("illegal configuration can not compute a moment")
        return

    x_loc = random.sample(range(0, n), v)
    x_el =[]
    x_val=[]
    for i in range (0,len(input_stream)):
        indices = [j for j, x in enumerate(x_el) if x == input_stream[i]]
        for index in indices:
            x_val[index]+=1

        if i in x_loc:
            x_el.append(input_stream[i])
            x_val.append(1)

    # in order to estimate the k-th moment the formula from lecture 13 slide 25 is used:
    moments = []
    for c in x_val:
        moments.append(n*(math.pow(c,k) -math.pow((c-1),k)))

    moment = np.mean(np.array(moments))
    return moment,x_val

### a)

In [93]:
stream = ['a', 'b', 'c', 'b', 'd', 'a', 'c', 'd', 'a', 'b', 'd', 'c', 'a', 'a', 'b']
ks = [1,2,3]
vs = [1,3,5,7,9]
n = 15

for k in ks:
    for v in vs:
        moment, x_val= compute_moment_ams(stream, n, k, v)
        print("Computed the following moment for: k="+str(k)+" , v="+str(v))
        print(moment)

Computed the following moment for: k=1 , v=1
15.0
Computed the following moment for: k=1 , v=3
15.0
Computed the following moment for: k=1 , v=5
15.0
Computed the following moment for: k=1 , v=7
15.0
Computed the following moment for: k=1 , v=9
15.0
Computed the following moment for: k=2 , v=1
15.0
Computed the following moment for: k=2 , v=3
75.0
Computed the following moment for: k=2 , v=5
57.0
Computed the following moment for: k=2 , v=7
66.42857142857143
Computed the following moment for: k=2 , v=9
65.0
Computed the following moment for: k=3 , v=1
285.0
Computed the following moment for: k=3 , v=3
225.0
Computed the following moment for: k=3 , v=5
483.0
Computed the following moment for: k=3 , v=7
272.14285714285717
Computed the following moment for: k=3 , v=9
215.0


### b)

In [113]:
estimation_moment_1,x_val = compute_moment_ams(stream, n, 3, 1)
estimation_moment_3,x_val = compute_moment_ams(stream, n, 3, 3)
estimation_moment_5,x_val = compute_moment_ams(stream, n, 3, 5)
estimation_moment_7,x_val = compute_moment_ams(stream, n, 3, 7)
estimation_moment_9,x_val = compute_moment_ams(stream, n, 3, 9)
estimation_moment_15,x_val = compute_moment_ams(stream, n, 3, 15)


x_val = np.array(x_val)
x_val = np.power(x_val,3)
exact_moment = np.sum(x_val)

print("The exact 3rd moment is:")
print(exact_moment)
print()
print("Using v=n we get a 3rd moment of:")
print(estimation_moment_15)
print("This gives us a defiation of:")
print(estimation_moment_15/exact_moment)
print()
print("Using v=9 we get a 3rd moment of:")
print(estimation_moment_9)
print("This gives us a defiation of:")
print(estimation_moment_9/exact_moment)

The exact 3rd moment is:
397

Using v=n we get a 3rd moment of:
243.0
This gives us a defiation of:
0.6120906801007556

Using v=9 we get a 3rd moment of:
275.0
This gives us a defiation of:
0.6926952141057935


### c)
Of course the estimation can only get better the more variables are used.
Hence, a bigger v leads to a better estimation.

# Exercise 4

We can use the general formula from above and solve it:
$f(c)=n*(c^4 -(c-1)^4)=n*(c^4 - ((c-1)^2 * (c-1)^2))=n*(c^4 - ((c^2-2c+1) * (c^2-2c+1))$
$=-n + 4 c n - 6 c^2 n + 4 c^3 n$

# Exercise 5